In [ ]:
import os
os.environ["OPENAI_API_KEY"] =
os.environ["HF_TOKEN"] = 

## 데이터셋 관련

In [ ]:
from datasets import load_dataset

In [ ]:
# https://huggingface.co/datasets/klue/klue/viewer/ynat
klue_ynat_train = load_dataset('klue','ynat',split='train')
klue_ynat_validation = load_dataset('klue','ynat',split='validation')

In [ ]:
# type(klue_ynat_train)
klue_ynat_train[0]

In [ ]:
vars(klue_ynat_train).keys()

In [ ]:
# vars(klue_ynat_train)['_info'] # 데이터셋이 가진 여러 정보
klue_ynat_train.features['label'].names # 설명 카테고리

In [ ]:
klue_ynat_train_data = klue_ynat_train.remove_columns(['guid','url','date'])
klue_ynat_validation_data = klue_ynat_validation.remove_columns(['guid','url','date'])

In [ ]:
klue_ynat_train_data, klue_ynat_validation_data

In [ ]:
# type(klue_ynat_train.features['label'])
klue_ynat_train.features['label']

In [ ]:
# type(klue_ynat_train.features['label'])
# klue_ynat_train.features['label'].int2str(6)
klue_ynat_train_data.features['label'].int2str(6)

### 데이터셋 분할

In [ ]:
klue_ynat_train_data_split = klue_ynat_train_data.train_test_split(test_size=10000, shuffle=True, seed=24)

klue_ynat_train_data_split['test']

## 모델 관련

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_id = 'klue/roberta-base'
out_features = len(klue_ynat_train.features['label'].names)

In [ ]:
# AutoModelForSequenceClassification.from_pretrained(model_id) # out_feature 랜덤하게 부여됨
model_ynat = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=7)

In [ ]:
model_ynat.state_dict()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
def tokenize_function(examples):
    return tokenizer(examples["title"], padding="max_length", truncation=True)

In [ ]:
train_dataset = klue_ynat_train_data.map(tokenize_function, batched=True)
validation_dataset = klue_ynat_validation_data.map(tokenize_function, batched=True)

In [ ]:
# type(train_dataset)
# train_dataset
train_dataset[0].keys()

In [ ]:
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(output_dir='./results_ynat'
                 , num_train_epochs=1
                 , per_device_train_batch_size=8
                 , per_device_eval_batch_size=8
                 , eval_strategy='epoch'
                 , learning_rate=0.001
                 , push_to_hub=False
                 , logging_steps=1
                 , report_to="none"  # WandB, TensorBoard 등 모두 비활성화
                                 )

In [ ]:
# 모델 평가 기준 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": (predictions == labels).mean()}

In [ ]:
trainer = Trainer(model = model_ynat
       , args = training_args
       , train_dataset = train_dataset
       , eval_dataset = validation_dataset
       , tokenizer = tokenizer
       , compute_metrics = compute_metrics)

In [ ]:
# 파인튜닝 시작
trainer.train()

## 모델 평가